In [1]:
import time
import yfinance as yf
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from datetime import datetime
import requests
import logging
import os
from dotenv import load_dotenv

# --- Load environment variables ---
load_dotenv()

API_KEY = os.getenv("API_KEY")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_PORT = os.getenv("DB_PORT")

logging.basicConfig(filename='trading.log', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# Use psycopg2 as the database driver to avoid asyncpg dependency issue
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# API Keys
ALPHA_VANTAGE_API_KEY = {API_KEY}
ALPHA_VANTAGE_URL = "https://www.alphavantage.co/query"

In [ ]:

def clean_stock_data(df):
    # Initial log of raw data shape
    logging.info(f'Initial data shape: {df.shape}')

    # Basic checks
    initial_rows = df.shape[0]

    # Remove rows with non-positive prices or volume
    df = df[(df['close_price'] > 0) & (df['volume'] > 0)]
    logging.info(f'Removed {initial_rows - df.shape[0]} rows with non-positive price or volume.')

    # Check for missing values and log them
    missing_before = df.isnull().sum().sum()
    if missing_before > 0:
        logging.info(f'Missing values before imputation: {missing_before}')

    # Impute missing values for prices and volumes using forward fill, then backward fill
    df['close_price'] = df['close_price'].ffill().bfill()
    df['volume'] = df['volume'].ffill().bfill()

    # Check if any missing values remain
    missing_after = df.isnull().sum().sum()
    if missing_after == 0:
        logging.info('All missing values handled.')
    else:
        logging.warning(f'Remaining missing values after imputation: {missing_after}')

    # Final log of cleaned data shape
    logging.info(f'Final data shape after cleaning: {df.shape}')

    return df

In [ ]:
#Validate DataFrame before calculations
def validate_dataframe(df):
    if df is None or df.empty:
        raise ValueError("Input DataFrame is empty or None.")
    if "close_price" not in df.columns:
        raise ValueError("Input DataFrame must contain 'close_price' column.")
    
def aggregate_data(df):
    validate_dataframe(df)
    logging.info("Aggregating data by date to prevent duplication.")
    aggregated_df = df.groupby('date').agg({
        'close_price': 'mean',
        'rsi': 'mean',
        'macd': 'mean',
        'macd_signal': 'mean'
    }).reset_index()
    logging.info(f"Aggregation completed. Resulting shape: {aggregated_df.shape}")
    return aggregated_df

# Function to calculate moving averages
def calculate_moving_averages(df, short_window=10, long_window=50):
    validate_dataframe(df)
    logging.info("Calculating moving averages with short_window=%d and long_window=%d", short_window, long_window)
    df[f"moving_avg_{short_window}"] = df["close_price"].rolling(window=short_window, min_periods=short_window).mean()
    df[f"moving_avg_{long_window}"] = df["close_price"].rolling(window=long_window, min_periods=long_window).mean()
    logging.info("Moving averages calculation completed")
    return df

# Function to calculate RSI
def calculate_rsi(df, period=14):
    validate_dataframe(df)
    logging.info("Calculating RSI with period %d", period)
    delta = df["close_price"].diff()

    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()

    rs = avg_gain / avg_loss.replace(0, 1)
    rsi = 100 - (100 / (1 + rs))

    df["rsi"] = rsi.clip(0, 100)
    logging.info("RSI calculation completed")
    return df

# Function to calculate MACD
def calculate_macd(df, short_period=12, long_period=26, signal_period=9):
    validate_dataframe(df)
    logging.info("Calculating MACD with short_period=%d, long_period=%d, signal_period=%d", short_period, long_period, signal_period)
    df["macd"] = df["close_price"].ewm(span=short_period, adjust=False).mean() - \
                 df["close_price"].ewm(span=long_period, adjust=False).mean()
    df["macd_signal"] = df["macd"].ewm(span=signal_period, adjust=False).mean()
    logging.info("MACD calculation completed")
    return df

# Function to apply all trend analysis
def apply_trend_analysis(df):
    validate_dataframe(df)
    logging.info("Applying trend analysis")
    df = calculate_moving_averages(df)
    df = calculate_rsi(df)
    df = calculate_macd(df)
    logging.info("Trend analysis completed")
    return df

In [ ]:
# Predefined list of stock symbols
stock_symbols = ["AAPL", "MSFT", "AMZN", "GOOGL", "NVDA"]

def insert_dim_stock(symbol):
    with engine.begin() as conn:
        result = conn.execute(text("SELECT stock_id FROM dim_stocks WHERE symbol = :symbol"), {"symbol": symbol}).fetchone()
        if result is None:
            conn.execute(text("INSERT INTO dim_stocks (symbol, company_name) VALUES (:symbol, :company_name)"),
                         {"symbol": symbol, "company_name": symbol})
            print(f"Inserted {symbol} into dim_stocks.")
        return conn.execute(text("SELECT stock_id FROM dim_stocks WHERE symbol = :symbol"), {"symbol": symbol}).scalar()

def insert_dim_date(date):
    date_str = date.strftime('%Y-%m-%d')
    with engine.begin() as conn:
        result = conn.execute(text("SELECT date_id FROM dim_dates WHERE full_date = :date"), {"date": date_str}).fetchone()
        if result is None:
            conn.execute(text("""
                INSERT INTO dim_dates (full_date, year, quarter, month, day, day_of_week, hour, am_pm)
                VALUES (:full_date, :year, :quarter, :month, :day, :day_of_week, :hour, :am_pm)
            """), {
                "full_date": date_str, "year": date.year, "quarter": (date.month - 1) // 3 + 1,
                "month": date.month, "day": date.day, "day_of_week": date.strftime('%A'),
                "hour": 0, "am_pm": "AM"
            })
        return conn.execute(text("SELECT date_id FROM dim_dates WHERE full_date = :date"), {"date": date_str}).scalar()

def insert_fact_trade(stock_id, date_id, trade_type, quantity, trade_price, volume):
    if trade_price <= 0 or volume <= 0:
        print(f'⚠ Skipping trade insertion due to invalid data: trade_price={trade_price}, volume={volume}')
        return
    
    with engine.begin() as conn:
        existing_trade = conn.execute(text("""
            SELECT 1 FROM fact_trades 
            WHERE stock_id = :stock_id AND date_id = :date_id AND trade_type = :trade_type 
            AND quantity = :quantity AND trade_price = :trade_price AND volume = :volume
        """), {
            "stock_id": stock_id, "date_id": date_id, "trade_type": trade_type,
            "quantity": quantity, "trade_price": trade_price, "volume": volume
        }).fetchone()
        
        if existing_trade:
            print(f'⚠ Skipping duplicate trade for stock_id={stock_id}, date_id={date_id}')
            return
        
        conn.execute(text("""
            INSERT INTO fact_trades (stock_id, date_id, trade_type, quantity, trade_price, volume)
            VALUES (:stock_id, :date_id, :trade_type, :quantity, :trade_price, :volume)
        """), {
            "stock_id": stock_id, "date_id": date_id, "trade_type": trade_type,
            "quantity": quantity, "trade_price": trade_price, "volume": volume
        })

def insert_fact_trends(stock_id, date_id, moving_avg_10, moving_avg_50, rsi, macd, macd_signal, price_change):
    with engine.begin() as conn:
        existing_trend = conn.execute(text("""
            SELECT 1 FROM fact_trends 
            WHERE stock_id = :stock_id AND date_id = :date_id
        """), {
            "stock_id": stock_id,
            "date_id": date_id
        }).fetchone()

        if existing_trend:
            print(f'⚠ Skipping duplicate trend data for stock_id={stock_id}, date_id={date_id}')
            return

        conn.execute(text("""
            INSERT INTO fact_trends (stock_id, date_id, moving_avg_10, moving_avg_50, rsi, macd, macd_signal, price_change)
            VALUES (:stock_id, :date_id, :moving_avg_10, :moving_avg_50, :rsi, :macd, :macd_signal, :price_change)
        """), {
            "stock_id": stock_id, "date_id": date_id,
            "moving_avg_10": moving_avg_10, "moving_avg_50": moving_avg_50,
            "rsi": rsi, "macd": macd, "macd_signal": macd_signal,
            "price_change": price_change
        })


def fetch_from_alpha_vantage(symbol):
    try:
        params = {
            "function": "TIME_SERIES_DAILY_ADJUSTED",
            "symbol": symbol,
            "apikey": ALPHA_VANTAGE_API_KEY,
            "outputsize": "full",
            "datatype": "json"
        }
        response = requests.get(ALPHA_VANTAGE_URL, params=params)
        data = response.json()
        
        if "Time Series (Daily)" not in data:
            print(f"⚠ Alpha Vantage data unavailable for {symbol}.")
            return None
        
        df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient="index")
        df.reset_index(inplace=True)
        df.rename(columns={
            "index": "Date",
            "1. open": "open_price",
            "4. close": "close_price",
            "2. high": "high",
            "3. low": "low",
            "6. volume": "volume"
        }, inplace=True)
        df["Date"] = pd.to_datetime(df["Date"])
        df.sort_values("Date", ascending=True, inplace=True)
        print(f"Alpha Vantage data fetched for {symbol}")
        return df
    except Exception as e:
        print(f"Alpha Vantage error for {symbol}: {e}")
        return None

def fetch_stock_data(stock_symbols, start="2024-01-01", end="2025-01-01"):
    all_stocks_data = {}
    for symbol in stock_symbols:
        try:
            stock = yf.download(symbol, start=start, end=end)
            if stock is None or stock.empty:
                print(f"⚠ No data from Yahoo Finance for {symbol}, trying Alpha Vantage...")
                stock = fetch_from_alpha_vantage(symbol)
                if stock is None or stock.empty:
                    print(f"No data available for {symbol}, skipping...")
                    continue
            
            print(f"Successfully fetched data for {symbol}: ")
            stock.rename(columns={"Open": "open_price", "Close": "close_price", "High": "high", "Low": "low", "Volume": "volume"}, inplace=True)
            stock = clean_stock_data(stock)
            stock["symbol"] = symbol
            stock.reset_index(inplace=True)

            # Apply calculations before trend analysis
            stock["daily_change"] = stock["close_price"].diff()

            # Apply trend analysis
            stock = apply_trend_analysis(stock)

            stock_id = insert_dim_stock(symbol)
            for _, row in stock.iterrows():
                if any(pd.isnull([row['open_price'], row['volume']])) or row['open_price'].item() <= 0 or row['volume'].item() <= 0:
                    print(f'⚠ Skipping invalid row for {symbol}: {row}')
                    continue
                date_value = row['Date'].iloc[0].date() if isinstance(row['Date'], pd.Series) else pd.to_datetime(row['Date']).date()
                date_id = insert_dim_date(date_value)
                trade_price = float(row['open_price'])
                volume = int(row['volume'])
                moving_avg_10 = float(row["moving_avg_10"].iloc[0]) if isinstance(row["moving_avg_10"], pd.Series) else float(row["moving_avg_10"])
                moving_avg_50 = float(row["moving_avg_50"].iloc[0]) if isinstance(row["moving_avg_50"], pd.Series) else float(row["moving_avg_50"])
                rsi = float(row["rsi"].iloc[0]) if isinstance(row["rsi"], pd.Series) else float(row["rsi"])
                macd = float(row["macd"].iloc[0]) if isinstance(row["macd"], pd.Series) else float(row["macd"])
                macd_signal = float(row["macd_signal"].iloc[0]) if isinstance(row["macd_signal"], pd.Series) else float(row["macd_signal"])
                price_change = float(row["daily_change"].iloc[0]) if isinstance(row["daily_change"], pd.Series) else float(row["daily_change"])

                insert_fact_trade(stock_id, date_id, 'BUY', 100, trade_price, volume)
                insert_fact_trends(stock_id, date_id, moving_avg_10, moving_avg_50, rsi, macd, macd_signal, price_change)


            all_stocks_data[symbol] = stock
            print(f"Data successfully loaded into PostgreSQL for {symbol}!")

        except Exception as e:
            print(f"Error fetching or inserting data for {symbol}: {e}")

    return all_stocks_data

def fetch_real_time_data(symbol, retry_attempts=3, retry_delay=5):
    stock = yf.Ticker(symbol)
    for attempt in range(retry_attempts):
        try:
            data = stock.history(period="1d")
            if not data.empty:
                break
        except Exception as e:
            print(f"⚠ Attempt {attempt + 1} failed for {symbol}: {e}")
            time.sleep(retry_delay)
    else:
        print(f"All {retry_attempts} attempts failed for {symbol}")
        return fetch_from_alpha_vantage(symbol)
    
    if data.empty:
        print(f"⚠ No real-time data from Yahoo Finance for {symbol}, trying Alpha Vantage...")
        return fetch_from_alpha_vantage(symbol)
    
    data.reset_index(inplace=True)
    data.rename(columns={"Open": "open_price", "Close": "close_price", "High": "high", "Low": "low", "Volume": "volume"}, inplace=True)
    print(f"Real-time data fetched for {symbol} from Yahoo Finance")
    return data

def update_real_time_data(stock_symbols, interval=60, duration=None):
    start_time = time.time()
    while duration is None or (time.time() - start_time) < duration:
        for symbol in stock_symbols:
            data = fetch_real_time_data(symbol)
            if data is not None and not data.empty:
                stock_id = insert_dim_stock(symbol)
                for _, row in data.iterrows():
                    if pd.isnull(row['open_price']) or pd.isnull(row['volume']) or row['open_price'] <= 0 or row['volume'] <= 0:
                        print(f'⚠ Skipping invalid row for {symbol}: {row}')
                        continue
                    date_value = pd.to_datetime(row['Date']).iloc[0].date() if isinstance(row['Date'], pd.Series) else pd.to_datetime(row['Date']).date()
                    date_id = insert_dim_date(date_value)
                    trade_price = float(row['open_price'])
                    volume = int(row['volume'])
                    insert_fact_trade(stock_id, date_id, 'BUY', 100, trade_price, volume)
                print(f"Real-time data updated for {symbol} in PostgreSQL")
        
        try:
            time.sleep(interval)
        except KeyboardInterrupt:
            print("⏹ Real-time data update stopped by user.")
            break

def fetch_trend_data(symbol):
    query = f"""
    SELECT d.full_date AS date, t.moving_avg_10, t.moving_avg_50, t.rsi, t.macd, t.macd_signal, 
           f.trade_price AS close_price, f.volume
    FROM fact_trends t
    JOIN dim_stocks s ON t.stock_id = s.stock_id
    JOIN dim_dates d ON t.date_id = d.date_id  -- JOIN to get full_date
    JOIN fact_trades f ON t.stock_id = f.stock_id AND t.date_id = f.date_id
    WHERE s.symbol = '{symbol}'
    ORDER BY d.full_date;
    """
    df = pd.read_sql(query, engine)

    # Rename & Convert Date Column
    df["date"] = pd.to_datetime(df["date"])  

    return df

In [5]:
all_stock_data = fetch_stock_data(stock_symbols)


# Print the first few rows of data for each stock
for symbol, df in all_stock_data.items():
    print(f"\n📊 Stock Data for {symbol}:")
    print(df[["Date", "close_price", "moving_avg_10", "daily_change"]].tail(50))

# Start real-time data update
update_real_time_data(stock_symbols, duration=3)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
/var/folders/_s/bwb12c8x3cnd5rpxh6xx0hl40000gn/T/ipykernel_21348/300356212.py:147: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  trade_price = float(row['open_price'])
/var/folders/_s/bwb12c8x3cnd5rpxh6xx0hl40000gn/T/ipykernel_21348/300356212.py:148: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  volume = int(row['volume'])


Successfully fetched data for AAPL: 
⚠ Skipping duplicate trend data for stock_id=6, date_id=256
⚠ Skipping duplicate trend data for stock_id=6, date_id=257
⚠ Skipping duplicate trend data for stock_id=6, date_id=258
⚠ Skipping duplicate trend data for stock_id=6, date_id=259
⚠ Skipping duplicate trend data for stock_id=6, date_id=260
⚠ Skipping duplicate trend data for stock_id=6, date_id=261
⚠ Skipping duplicate trend data for stock_id=6, date_id=262
⚠ Skipping duplicate trend data for stock_id=6, date_id=263
⚠ Skipping duplicate trend data for stock_id=6, date_id=264
⚠ Skipping duplicate trend data for stock_id=6, date_id=265
⚠ Skipping duplicate trend data for stock_id=6, date_id=266
⚠ Skipping duplicate trend data for stock_id=6, date_id=267
⚠ Skipping duplicate trend data for stock_id=6, date_id=268
⚠ Skipping duplicate trend data for stock_id=6, date_id=269
⚠ Skipping duplicate trend data for stock_id=6, date_id=270
⚠ Skipping duplicate trend data for stock_id=6, date_id=271
⚠ S

[*********************100%***********************]  1 of 1 completed

⚠ Skipping duplicate trend data for stock_id=6, date_id=483
⚠ Skipping duplicate trend data for stock_id=6, date_id=484
⚠ Skipping duplicate trend data for stock_id=6, date_id=485
⚠ Skipping duplicate trend data for stock_id=6, date_id=486
⚠ Skipping duplicate trend data for stock_id=6, date_id=487
⚠ Skipping duplicate trend data for stock_id=6, date_id=488
⚠ Skipping duplicate trend data for stock_id=6, date_id=489
⚠ Skipping duplicate trend data for stock_id=6, date_id=490
⚠ Skipping duplicate trend data for stock_id=6, date_id=491
⚠ Skipping duplicate trend data for stock_id=6, date_id=492
⚠ Skipping duplicate trend data for stock_id=6, date_id=493
⚠ Skipping duplicate trend data for stock_id=6, date_id=494
⚠ Skipping duplicate trend data for stock_id=6, date_id=495
⚠ Skipping duplicate trend data for stock_id=6, date_id=496
⚠ Skipping duplicate trend data for stock_id=6, date_id=497
⚠ Skipping duplicate trend data for stock_id=6, date_id=498
⚠ Skipping duplicate trend data for stoc

...

⚠ Skipping duplicate trend data for stock_id=9, date_id=269
⚠ Skipping duplicate trend data for stock_id=9, date_id=270
⚠ Skipping duplicate trend data for stock_id=9, date_id=271
⚠ Skipping duplicate trend data for stock_id=9, date_id=272
⚠ Skipping duplicate trend data for stock_id=9, date_id=273
⚠ Skipping duplicate trend data for stock_id=9, date_id=274
⚠ Skipping duplicate trend data for stock_id=9, date_id=275
⚠ Skipping duplicate trend data for stock_id=9, date_id=276
⚠ Skipping duplicate trend data for stock_id=9, date_id=277
⚠ Skipping duplicate trend data for stock_id=9, date_id=278
⚠ Skipping duplicate trend data for stock_id=9, date_id=279
⚠ Skipping duplicate trend data for stock_id=9, date_id=280
⚠ Skipping duplicate trend data for stock_id=9, date_id=281
⚠ Skipping duplicate trend data for stock_id=9, date_id=282
⚠ Skipping duplicate trend data for stock_id=9, date_id=283
⚠ Skipping duplicate trend data for stock_id=9, date_id=284
⚠ Skipping duplicate trend data for stoc

[*********************100%***********************]  1 of 1 completed

⚠ Skipping duplicate trend data for stock_id=9, date_id=505
⚠ Skipping duplicate trend data for stock_id=9, date_id=506
⚠ Skipping duplicate trend data for stock_id=9, date_id=507
Data successfully loaded into PostgreSQL for GOOGL!
Successfully fetched data for NVDA: 
⚠ Skipping duplicate trend data for stock_id=10, date_id=256
⚠ Skipping duplicate trend data for stock_id=10, date_id=257
⚠ Skipping duplicate trend data for stock_id=10, date_id=258
⚠ Skipping duplicate trend data for stock_id=10, date_id=259
⚠ Skipping duplicate trend data for stock_id=10, date_id=260
⚠ Skipping duplicate trend data for stock_id=10, date_id=261
⚠ Skipping duplicate trend data for stock_id=10, date_id=262
⚠ Skipping duplicate trend data for stock_id=10, date_id=263
⚠ Skipping duplicate trend data for stock_id=10, date_id=264
⚠ Skipping duplicate trend data for stock_id=10, date_id=265
⚠ Skipping duplicate trend data for stock_id=10, date_id=266
⚠ Skipping duplicate trend data for stock_id=10, date_id=267


/var/folders/_s/bwb12c8x3cnd5rpxh6xx0hl40000gn/T/ipykernel_21348/300356212.py:147: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  trade_price = float(row['open_price'])
/var/folders/_s/bwb12c8x3cnd5rpxh6xx0hl40000gn/T/ipykernel_21348/300356212.py:148: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  volume = int(row['volume'])


⚠ Skipping duplicate trend data for stock_id=10, date_id=290
⚠ Skipping duplicate trend data for stock_id=10, date_id=291
⚠ Skipping duplicate trend data for stock_id=10, date_id=292
⚠ Skipping duplicate trend data for stock_id=10, date_id=293
⚠ Skipping duplicate trend data for stock_id=10, date_id=294
⚠ Skipping duplicate trend data for stock_id=10, date_id=295
⚠ Skipping duplicate trend data for stock_id=10, date_id=296
⚠ Skipping duplicate trend data for stock_id=10, date_id=297
⚠ Skipping duplicate trend data for stock_id=10, date_id=298
⚠ Skipping duplicate trend data for stock_id=10, date_id=299
⚠ Skipping duplicate trend data for stock_id=10, date_id=300
⚠ Skipping duplicate trend data for stock_id=10, date_id=301
⚠ Skipping duplicate trend data for stock_id=10, date_id=302
⚠ Skipping duplicate trend data for stock_id=10, date_id=303
⚠ Skipping duplicate trend data for stock_id=10, date_id=304
⚠ Skipping duplicate trend data for stock_id=10, date_id=305
⚠ Skipping duplicate tre